This notebook implements finitary versions of the approach to the probabilities of conditionals from Bas van Fraassen's 1976 paper "Probabilities of conditionals" (https://link.springer.com/chapter/10.1007/978-94-010-1853-1_10) (cf. Section 5 of http://paolosantorio.net/KS-NASSLLI2018.pdf).

# Outline

### 1. Preliminaries

### 2. Semantics

### 3. Probability

> 3.1. Fair die case

> 3.2. Biased die case

### 4. Modified semantics

> 4.1. McGee-style counterexample to Modus Ponens

### 5. A second version of the modified semantics

> 5.1. Differences with non-Boolean antecedents

# 1. Preliminaries

In [205]:
def perms(urn, num_to_draw = None):
    """
    Returns all permutations of subsets of urn of size num_to_draw (default is len(urn))
    """
    if num_to_draw == None:
        num_to_draw = len(urn)

    return _perms(urn,num_to_draw,[])

def _perms(urn, num_to_draw, sequence):

    if num_to_draw == 1:
        for w in urn:
            yield tuple(sequence + [w])

    if num_to_draw > 1:
        for w in urn:
            yield from _perms([v for v in urn if not v==w], num_to_draw-1, sequence + [w])

In [206]:
for seq in perms(['x','y','z']):
    print(seq)

('x', 'y', 'z')
('x', 'z', 'y')
('y', 'x', 'z')
('y', 'z', 'x')
('z', 'x', 'y')
('z', 'y', 'x')


In [207]:
def permutations_of_ne_subsets(urn):
    """
    Returns all permutations of nonempty subsets of urn
    """

    for n in range(len(urn)+1):

        yield from perms(urn,n)

In [208]:
for seq in permutations_of_ne_subsets(['x','y','z']):
    print(seq)

('x',)
('y',)
('z',)
('x', 'y')
('x', 'z')
('y', 'x')
('y', 'z')
('z', 'x')
('z', 'y')
('x', 'y', 'z')
('x', 'z', 'y')
('y', 'x', 'z')
('y', 'z', 'x')
('z', 'x', 'y')
('z', 'y', 'x')


# 2. Semantics

A *model* is a pair $\mathcal{M}=(W,V)$ of a nonempty set $W$ and a propositional varluation $V:\mathsf{Prop}\to \wp(W)$.

Let $W'$ be the set of all permutations of nonempty subsets of $W$. We define truth of a formula $\varphi$ at a permutation $\langle w_1,\dots,w_n\rangle\in W'$ as follows:

1. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash p$ iff $w_1\in V(p)$;

2. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \neg\varphi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\nvDash \varphi$;

3. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi\wedge\psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$ and $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$;

4. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi\vee\psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$ or $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi$;

5. $\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi > \psi$ iff $\mathrm{min}(\{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle\vDash \varphi \})\subseteq \{i \in\mathbb{N}\mid \mathcal{M},\langle w_i,\dots,w_n\rangle\vDash \psi \}$,

where $\langle w_i,\dots,w_n\rangle$ is the final segment of $\langle w_1,\dots,w_n\rangle$ beginning with $w_i$.

We call $\{\sigma\in W'\mid \mathcal{M},\sigma\vDash\varphi\}$ the *extension* of $\varphi$.

In [209]:
def main_connective(formula):
    """
    Returns the main connective of the formula (or "prop" if s is a propositional variable) and arguments to main connective (or propositional variable)
    """

    formula = formula.replace(" ", "")
        
    if len(formula) == 1:
        return "prop", formula

    if formula.startswith("~"):
        formula = formula.removeprefix("~")
        return "~", formula

    if formula.startswith("("):
            
        leftparen = 0
        rightparen = 0
        slength = len(formula)

        for n in range(slength):

            if formula[n] == "(":
                leftparen +=1

            if formula[n] == ")":
                rightparen +=1

            if leftparen - rightparen == 1:
                
                if formula[n] == "^":
                    return "^", formula[1:n], formula[n+1:-1]

                if formula[n] == "v":
                    return "v", formula[1:n], formula[n+1:-1]
                
                if formula[n] == '>':
                    return '>', formula[1:n], formula[n+1:-1]     

    return None

In [210]:
def extension(formula, sequences, valuation):
    """
    Given a formula, a set of sequences of worlds, and a valuation mapping each proposition letter to a set of worlds,
    returns the extension of the formula as a set of sequences
    """
    
    main_connec = main_connective(formula)

    if main_connec[0] == 'prop':

        return set([sequence for sequence in sequences if sequence[0] in valuation[main_connec[1]]])

    if main_connec[0] == '~':
        inner_ext = set(extension(main_connec[1], sequences,valuation))
        return set([w for w in sequences if not w in inner_ext])

    if main_connec[0] == '^':
        return extension(main_connec[1],sequences,valuation).intersection(extension(main_connec[2],sequences,valuation))

    if main_connec[0] == 'v':
        return extension(main_connec[1],sequences,valuation).union(extension(main_connec[2],sequences,valuation))

    if main_connec[0] == '>':

        ant_ext = extension(main_connec[1], sequences, valuation)
        cons_ext = extension(main_connec[2], sequences, valuation)

        ext = [seq for seq in sequences]

        for sequence in sequences:
            for idx, world in enumerate(sequence):
                if sequence[idx::] in ant_ext:
                    if not sequence[idx::] in cons_ext:

                        ext.remove(sequence)
                        break
                        
                    else:
                        break

        return set(ext)

In [211]:
seqs = list(permutations_of_ne_subsets(['x','y','z']))
print(seqs)

[('x',), ('y',), ('z',), ('x', 'y'), ('x', 'z'), ('y', 'x'), ('y', 'z'), ('z', 'x'), ('z', 'y'), ('x', 'y', 'z'), ('x', 'z', 'y'), ('y', 'x', 'z'), ('y', 'z', 'x'), ('z', 'x', 'y'), ('z', 'y', 'x')]


In [212]:
extension('p',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y'),
 ('z', 'y'),
 ('z', 'y', 'x')}

In [213]:
extension('q',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z'),
 ('y', 'z'),
 ('y', 'z', 'x')}

In [214]:
extension('(p>q)',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z')}

In [215]:
extension('(q>p)',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y')}

In [216]:
extension('(p>(q>p))',seqs,{'p': set({'x','z'}), 'q': set({'x','y'})})

{('x',),
 ('x', 'y'),
 ('x', 'y', 'z'),
 ('x', 'z'),
 ('x', 'z', 'y'),
 ('y',),
 ('y', 'x'),
 ('y', 'x', 'z'),
 ('y', 'z'),
 ('y', 'z', 'x'),
 ('z',),
 ('z', 'x'),
 ('z', 'x', 'y')}

# 3. Probability of formulas

Given a finite model $\mathcal{M}=(W,V)$, we encode a probability distribution as a dictionary that assigns to each $w\in W$ a number in $[0,1]$ such that the probabilities sum to 1.

In [217]:
mu = dict()
mu['x'] = .4
mu['y'] = .35
mu['z'] = .25

Next we assign probabilities to permutations of $W$. Note: we do not assign probabilities to permutations of proper subsets of $W$, since this will not be needed in the following.

We take the probability of a permutation of $W$ to be the probability of drawing that permutation via sampling without replacement from $W$ according to the given probability distribution.

In [218]:
def prob_without_replacement(sequence,world_prob_dist):
    """
    Given a sequence of worlds and a probability distribution over worlds,
    returns the probability of the sequence under sampling without replacement
    """

    world = sequence[0]

    if len(sequence) == 1:
        return world_prob_dist[world]

    new_world_prob_dist = dict()

    for w in world_prob_dist.keys():
        if not w == sequence[0]:

            new_world_prob_dist[w] =  world_prob_dist[w] /(1-world_prob_dist[world])

    return world_prob_dist[world] * prob_without_replacement(sequence[1:],new_world_prob_dist)

In [219]:
prob_without_replacement(['x','y','z'], mu)

0.2333333333333334

In [220]:
def sequence_prob_dist(world_prob_dist):
    """
    Given a probability distribution over worlds,
    returns the probability distribution over permutations of the set of worlds
    given by sampling without replacement
    """

    sequence_prob_dist = dict()

    for sequence in perms(world_prob_dist.keys()):

        sequence_prob_dist[sequence] =  prob_without_replacement(sequence,world_prob_dist)

    return sequence_prob_dist

In [221]:
sequence_prob_dist(mu)

{('x', 'y', 'z'): 0.2333333333333334,
 ('x', 'z', 'y'): 0.16666666666666674,
 ('y', 'x', 'z'): 0.2153846153846154,
 ('y', 'z', 'x'): 0.13461538461538458,
 ('z', 'x', 'y'): 0.1333333333333333,
 ('z', 'y', 'x'): 0.11666666666666663}

Given a model $\mathcal{M}=(W,V)$ and probabilities encoded by $\mu:W\to[0,1]$, let $\mu^+:\wp(W)\to [0,1]$ be defined as usual by $\mu^+(A)=\underset{w\in A}{\sum}\mu(w)$.

Then we take the probability of a formula $\varphi$ to be $\mu^+(\{\sigma\in permutations(W)\mid \mathcal{M},\sigma\vDash\varphi\})$.

In [222]:
def prob_of_formula(formula, world_prob_dist, valuation):
    """
    Given a formula, probability distribution over worlds, and propositional valuation,
    returns the probability of the formula
    """

    worlds = world_prob_dist.keys()

    sequences = list(permutations_of_ne_subsets(worlds))

    ext = extension(formula,sequences,valuation)

    seq_prob_dist = sequence_prob_dist(world_prob_dist)

    full_perms_ext = [seq for seq in perms(worlds) if seq in ext]

    return sum([seq_prob_dist[seq] for seq in full_perms_ext])

## 3.1. Fair die case

In [223]:
fair_die = dict()

for n in range(1,7):
    fair_die[n] = 1/6

print(fair_die)


{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


In [224]:
val = {'l': set({1,2,3}), 'h': set({4,5,6}), 'e': set({2,4,6})}

In [225]:
prob_of_formula('(l>e)', fair_die, val)

0.3333333333333324

In [226]:
prob_of_formula('(e^l)', fair_die, val) / prob_of_formula('l', fair_die, val)

0.33333333333333365

In [227]:
prob_of_formula('(h>e)', fair_die, val)

0.6666666666666646

In [228]:
prob_of_formula('(e^h)', fair_die, val) / prob_of_formula('h', fair_die, val)

0.6666666666666669

In [229]:
prob_of_formula('((l>e)^(h>e))', fair_die ,val) 

0.22222222222222168

By contrast, according to Ciardelli and Ommunndsen (2022, Sec. 5) (https://onlinelibrary.wiley.com/doi/abs/10.1111/nous.12437), $(l>e)\wedge (h>e)$ is equivalent to $e$ and hence should have probability .5. By this reasoning, the probability of $(l>e)\wedge (h>e)$ should also be .5 in the following biased die case. 

## 3.2. Biased die case

In [230]:
biased_die = dict()

biased_die[1] = 4/24
biased_die[2] = 1/24
biased_die[3] = 4/24
biased_die[4] = 6/24
biased_die[5] = 4/24
biased_die[6] = 5/24

In [231]:
prob_of_formula('e', biased_die, val)

0.5000000000000006

In [232]:
prob_of_formula('(l>e)', biased_die, val)

0.11111111111111105

In [233]:
prob_of_formula('(e^l)', biased_die, val) / prob_of_formula('l', biased_die, val)

0.11111111111111112

In [234]:
prob_of_formula('(h>e)', biased_die, val)

0.7333333333333321

In [235]:
prob_of_formula('(e^h)', biased_die, val) / prob_of_formula('h', biased_die, val)

0.733333333333335

In [236]:
prob_of_formula('((l>e)^(h>e))', biased_die, val)

0.08148148148148146

As noted above, the reasoning of Ciardelli and Ommunndsen (2022, Sec. 5) implies that the probability of $(l>e)\wedge (h>e)$ should be .5 in this case.

# 4. Modified semantics

We now give a modified semantics that validates $p> (q > p)$, unlike the semantics of Section 2.

This semantics is intended only for conditionals $\varphi>\psi$ where $\varphi$ is Boolean (see Section 5.1 for a reason why). A more complicated definition (perhaps the one in Section 5) should be used when $\varphi$ contains conditionals.

The new clause for $>$ is:

$\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi > \psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle +\varphi \vDash \psi$ where $\langle w_1,\dots,w_n\rangle +\varphi$ is the sequence obtained from $\langle w_1,\dots,w_n\rangle$ by dropping those $w_i$ such that $\mathcal{M},\langle w_i,\dots,w_n\rangle\nvDash \varphi$.

Note that $\langle w_1,\dots,w_n\rangle +\varphi$ may be the empty sequence, and we stipulate that $\mathcal{M},\langle \rangle \vDash \psi$.


In [237]:
def extension2(formula, sequences, valuation):
    """
    Given a formula, a set of sequences of worlds, and a valuation mapping each proposition letter to a set of worlds,
    returns the extension of the formula as a set of sequences according to the modified semantics
    """
    
    main_connec = main_connective(formula)

    if main_connec[0] == 'prop':

        return set([sequence for sequence in sequences if sequence[0] in valuation[main_connec[1]]])

    if main_connec[0] == '~':
        inner_ext = set(extension2(main_connec[1], sequences,valuation))
        return set([w for w in sequences if not w in inner_ext])

    if main_connec[0] == '^':
        return extension2(main_connec[1],sequences,valuation).intersection(extension2(main_connec[2],sequences,valuation))

    if main_connec[0] == 'v':
        return extension2(main_connec[1],sequences,valuation).union(extension2(main_connec[2],sequences,valuation))

    if main_connec[0] == '>':

        ant_ext = extension2(main_connec[1], sequences, valuation)
        cons_ext = extension2(main_connec[2], sequences, valuation)

        ext = [seq for seq in sequences]

        for sequence in sequences:
                
            restricted_sequence = tuple()

            for idx, world in enumerate(sequence):

                if sequence[idx::] in ant_ext:
                    restricted_sequence = restricted_sequence + (world,)

            if len(restricted_sequence) > 0 and not restricted_sequence in cons_ext:
                ext.remove(sequence)

        return set(ext)

In [238]:
def prob_of_formula2(formula, world_prob_dist, valuation):
    """
    Given a formula, probability distribution over worlds, and propositional valuation,
    returns the probability of the formula according to the modified semantics
    """

    worlds = world_prob_dist.keys()

    sequences = list(permutations_of_ne_subsets(worlds))

    ext = extension2(formula,sequences,valuation)

    seq_prob_dist = sequence_prob_dist(world_prob_dist)

    full_perms_ext = [seq for seq in perms(worlds) if seq in ext]

    return sum([seq_prob_dist[seq] for seq in full_perms_ext])

We get the same verdicts on the formulas we tested for the fair and biased die cases.

In [239]:
print(prob_of_formula2('(l>e)', fair_die, val))
print(prob_of_formula2('((l>e)^(h>e))', fair_die ,val))
print(prob_of_formula2('(l>e)', biased_die, val))
print(prob_of_formula2('((l>e)^(h>e))', biased_die, val))

0.3333333333333324
0.22222222222222168
0.11111111111111105
0.08148148148148146


## 4.1. McGee-style counterexample to modus ponens

This example is from Section 4.2 of Paolo Santorio's (2021) "Trivializing Informational Consequence" (http://paolosantorio.net/tic.pdf), based on McGee's (1985) "A Counterexample to Modus Ponens" (https://www.pdcnet.org/jphil/content/jphil_1985_0082_0009_0462_0471). 

Suppose you roll a fair 6-sided die.

What is the probability that *if the die lands on an even number, then if it doesn't land on 2 or 4, then it lands on 6*? (Intuitively, 1.)

What is the probability that *if the die doesn't land on 2 or 4, then it lands on 6*? (Intuitively, .25.)

According to McGee and Santorio, you can assign greater probability to $e \wedge (e > (\neg(t \vee f) > s))$ than to $\neg( t \vee  f) > s$. The approach of Section 4 agrees, while the approach of Section 2 disagrees.


In [240]:
seqs = list(permutations_of_ne_subsets([1,2,3,4,5,6]))
val2 = {'e': set({2,4,6}), 't': set({2}), 'f': set({4}), 's': set({6})}

print("Total number of sequences:", len(seqs))
print("Number of sequences in which (e > (~(t v f) > s)) is true according to Section 2:", len(extension('(e > (~(t v f) > s))', seqs, val2))) 
print("Number of sequences in which (e > (~(t v f) > s)) is true according to Section 4:", len(extension2('(e > (~(t v f) > s))', seqs, val2))) 


Total number of sequences: 1956
Number of sequences in which (e > (~(t v f) > s)) is true according to Section 2: 1134
Number of sequences in which (e > (~(t v f) > s)) is true according to Section 4: 1956


In [241]:
print("Probability of (e > (~(t v f) > s)) according to Section 2:", prob_of_formula('(e > (~(t v f) > s))', fair_die, val2))
print("Probability of (e > (~(t v f) > s)) according to Section 4:", prob_of_formula2('(e > (~(t v f) > s))', fair_die, val2))

Probability of (e > (~(t v f) > s)) according to Section 2: 0.566666666666665
Probability of (e > (~(t v f) > s)) according to Section 4: 0.9999999999999968


In [242]:
# The Modus Ponens inference according to the Section 2 semantics:

print("Probability of (e ^ (e > (~(t v f) > s))) according to Section 2:", prob_of_formula('(e ^ (e > (~(t v f) > s)))', fair_die, val2))
print("Probability of ((~t ^ ~f) > s) according to Section 2:", prob_of_formula('((~t ^ ~f) > s)', fair_die, val2))

Probability of (e ^ (e > (~(t v f) > s))) according to Section 2: 0.24999999999999936
Probability of ((~t ^ ~f) > s) according to Section 2: 0.24999999999999936


In [243]:
# The Modus Ponens inference according to the Section 4 semantics:

print("Probability of (e ^ (e > (~(t v f) > s))) according to Section 4:", prob_of_formula2('(e ^ (e > (~(t v f) > s)))', fair_die, val2))
print("Probability of ((~t ^ ~f) > s) according to Section 4:", prob_of_formula2('((~t ^ ~f) > s)', fair_die, val2))

Probability of (e ^ (e > (~(t v f) > s))) according to Section 4: 0.4999999999999985
Probability of ((~t ^ ~f) > s) according to Section 4: 0.24999999999999936


# 5. A second version of the modified semantics

Here is a variant of the Section 4 semantics, which is equivalent to that semantics if $\varphi$ is Boolean but not in general:

$\mathcal{M},\langle w_1,\dots,w_n\rangle\vDash \varphi > \psi$ iff $\mathcal{M},\langle w_1,\dots,w_n\rangle \times\varphi \vDash \psi$ where $\langle w_1,\dots,w_n\rangle \times\varphi$ is the sequence obtained from $\langle w_1,\dots,w_n\rangle$ by the following inductive construction (where $\tau+\sigma$ is the concatation of the sequences $\tau$ and $\sigma$):

1. if $\mathcal{M},\langle w_n\rangle\vDash \varphi$, let $\sigma_1=\langle w_n\rangle$; otherwise $\sigma_1=\langle \rangle$;
2. if $\mathcal{M},\langle w_{n-1}\rangle + \sigma_1 \vDash \varphi$, let $\sigma_2=\langle w_{n-1}\rangle + \sigma_1$; otherwise $\sigma_2=\sigma_1$;
2. if $\mathcal{M},\langle w_{n-2}\rangle + \sigma_2 \vDash \varphi$, let $\sigma_3=\langle w_{n-2}\rangle + \sigma_2$; otherwise $\sigma_3=\sigma_2$;
4. etc.

Then $\langle w_1,\dots,w_n\rangle \times\varphi=\sigma_n$.

As before, $\langle w_1,\dots,w_n\rangle \times\varphi$ may be the empty sequence, and we stipulate that $\mathcal{M},\langle \rangle \vDash \psi$.

In [244]:
def extension3(formula, sequences, valuation):
    """
    Given a formula, a set of sequences of worlds, and a valuation mapping each proposition letter to a set of worlds,
    returns the extension of the formula as a set of sequences according to the second version of the modified semantics
    """
    
    main_connec = main_connective(formula)

    if main_connec[0] == 'prop':

        return set([sequence for sequence in sequences if sequence[0] in valuation[main_connec[1]]])

    if main_connec[0] == '~':
        inner_ext = set(extension3(main_connec[1], sequences,valuation))
        return set([w for w in sequences if not w in inner_ext])

    if main_connec[0] == '^':
        return extension3(main_connec[1],sequences,valuation).intersection(extension3(main_connec[2],sequences,valuation))

    if main_connec[0] == 'v':
        return extension3(main_connec[1],sequences,valuation).union(extension3(main_connec[2],sequences,valuation))

    if main_connec[0] == '>':

        ant_ext = extension3(main_connec[1], sequences, valuation)
        cons_ext = extension3(main_connec[2], sequences, valuation)

        ext = [seq for seq in sequences]

        for sequence in sequences:

            reversed_sequence = tuple(reversed(sequence))
                
            restricted_sequence = tuple()

            for world in reversed_sequence:

                if (world,) + restricted_sequence in ant_ext:
                    restricted_sequence = (world,) + restricted_sequence

            if len(restricted_sequence) > 0 and not restricted_sequence in cons_ext:
                ext.remove(sequence)

        return set(ext)

In [245]:
def prob_of_formula3(formula, world_prob_dist, valuation):
    """
    Given a formula, probability distribution over worlds, and propositional valuation,
    returns the probability of the formula according to the modified semantics
    """

    worlds = world_prob_dist.keys()

    sequences = list(permutations_of_ne_subsets(worlds))

    ext = extension3(formula,sequences,valuation)

    seq_prob_dist = sequence_prob_dist(world_prob_dist)

    full_perms_ext = [seq for seq in perms(worlds) if seq in ext]

    return sum([seq_prob_dist[seq] for seq in full_perms_ext])

The McGee-style counterexample to Modus Ponens works just as before.

In [246]:
print("Probability of (e > (~(t v f) > s)) according to Section 5:", prob_of_formula3('(e > (~(t v f) > s))', fair_die, val2))
print("Probability of (e ^ (e > (~(t v f) > s))) according to Section 5:", prob_of_formula3('(e ^ (e > (~(t v f) > s)))', fair_die, val2))
print("Probability of ((~t ^ ~f) > s) according to Section 5:", prob_of_formula3('((~t ^ ~f) > s)', fair_die, val2))

Probability of (e > (~(t v f) > s)) according to Section 5: 0.9999999999999968
Probability of (e ^ (e > (~(t v f) > s))) according to Section 5: 0.4999999999999985
Probability of ((~t ^ ~f) > s) according to Section 5: 0.24999999999999936


## 5.1. Differences with non-Boolean antecedents

The semantics of Sections 4 and 5 are equivalent for conditionals with Boolean antecedents, but here we consider differences when the antecedent is non-Boolean.

First, the semantics of Section 4 invalidates the identity principle $\varphi>\varphi$ for some exotic cases where $\varphi$ contains conditionals.... By contrast, it's not hard to see that $\varphi>\varphi$ is valid according to the semantics of Section 5.

In [247]:
frml = '(f^((t>f)>t))'
print("Total number of sequences:", len(seqs))
print("Number of sequences in which (frml > frml) is true according to Section 4:", 
      len(extension2('(' + frml + ">" + frml + ')', seqs, val2))) 
print("Number of sequences in which (frml > frml) is true according to Section 5:", 
      len(extension3('(' + frml + ">" + frml + ')', seqs, val2))) 

Total number of sequences: 1956
Number of sequences in which (frml > frml) is true according to Section 4: 1695
Number of sequences in which (frml > frml) is true according to Section 5: 1956


Also interesting are the following conflicting probability verdicts.

In [248]:
print("Probability of ((e>t)>t) according to Section 4:", 
      prob_of_formula2('((e>t)>t)', fair_die, val2))
print("Probability of t contiditional on (e>t) according to Section 4:", 
      prob_of_formula2('((e>t)^t)', fair_die, val2) / prob_of_formula2('(e>t)', fair_die, val2))
print("")
print("Probability of ((e>t)>t) according to Section 5:", 
      prob_of_formula3('((e>t)>t)', fair_die, val2))
print("Probability of t contiditional on (e>t) according to Section 5:", 
      prob_of_formula3('((e>t)^t)', fair_die, val2) / prob_of_formula3('(e>t)', fair_die, val2))

Probability of ((e>t)>t) according to Section 4: 0.4999999999999985
Probability of t contiditional on (e>t) according to Section 4: 0.5000000000000003

Probability of ((e>t)>t) according to Section 5: 0.24999999999999936
Probability of t contiditional on (e>t) according to Section 5: 0.5000000000000003


In [249]:
print("Probability of ((e>t)>e) according to Section 4:", 
      prob_of_formula2('((e>t)>e)', fair_die, val2))
print("Probability of e conditional on (e>t) according to Section 4:", 
      prob_of_formula2('((e>t)^e)', fair_die, val2) / prob_of_formula2('(e>t)', fair_die, val2))
print("")
print("Probability of ((e>t)>e) according to Section 5:", 
      prob_of_formula3('((e>t)>e)', fair_die, val2))
print("Probability of e contiditional on (e>t) according to Section 5:", 
      prob_of_formula3('((e>t)^e)', fair_die, val2) / prob_of_formula3('(e>t)', fair_die, val2))

Probability of ((e>t)>e) according to Section 4: 0.4999999999999985
Probability of e conditional on (e>t) according to Section 4: 0.5000000000000003

Probability of ((e>t)>e) according to Section 5: 0.24999999999999936
Probability of e contiditional on (e>t) according to Section 5: 0.5000000000000003
